# EDA : Évolution Climatique des Grandes Villes
**Problématique** : 
- Comment évolue le climat dans plusieurs grandes villes ?
- Peut-on classer les villes selon leur stabilité, niveau de chaleur ou variabilité ?

**Sources** : Données OpenWeather (actuelles) et Open-Meteo (historiques)

In [1]:
# Import des librairies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline

# Configuration
plt.style.use('ggplot')
sns.set_palette("husl")
pd.set_option('display.max_columns', 50)

Matplotlib is building the font cache; this may take a moment.


In [5]:
# Chargement des données
dim_ville = pd.read_csv("data/star_schema/dim_ville.csv")
fact_weather = pd.read_csv("data/star_schema/fact_weather.csv")

# Fusion et préparation
df = pd.merge(fact_weather, dim_ville, on='ville_id')
df['date'] = pd.to_datetime(df['date'])
df['annee'] = df['date'].dt.year
df['mois'] = df['date'].dt.month_name()

# Ajout indicateurs climatiques
df['amplitude_thermique'] = df['temp_max'] - df['temp_min']

FileNotFoundError: [Errno 2] No such file or directory: 'data/star_schema/dim_ville.csv'

## 1. Stabilité Climatique (Variabilité)

In [ ]:
# Calcul de la variabilité
stabilite = df.groupby('ville').agg({
    'temperature': ['mean', 'std'],
    'amplitude_thermique': 'mean'
}).sort_values(('temperature', 'std'))

# Visualisation
plt.figure(figsize=(12, 6))
sns.barplot(x=stabilite.index, y=stabilite[('temperature', 'std')])
plt.title('Variabilité Climatique (Écart-type des Températures)')
plt.ylabel('Écart-type (°C)')
plt.xticks(rotation=45)
plt.show()

display(Markdown("### Classement par Stabilité (du plus stable au plus variable):"))
display(stabilite)

## 2. Niveau de Chaleur Moyen

In [ ]:
# Températures moyennes annuelles
chaleur = df.groupby(['ville', 'annee'])['temperature'].mean().unstack()

# Visualisation
plt.figure(figsize=(12, 6))
chaleur.mean(axis=1).sort_values().plot(kind='barh', color='darkred')
plt.title('Température Moyenne sur la Période')
plt.xlabel('Température (°C)')
plt.grid(True)
plt.show()

display(Markdown("### Évolution Annuelle:"))
chaleur.T.plot(figsize=(12, 6), title='Évolution des Températures Moyennes')
plt.ylabel('Température (°C)')
plt.show()

## 3. Analyse des Tendances

In [ ]:
# Calcul des tendances linéaires
from scipy.stats import linregress

resultats = []
for ville in df['ville'].unique():
    subset = df[df['ville'] == ville]
    slope, _, _, _, _ = linregress(subset['annee'], subset['temperature'])
    resultats.append({'Ville': ville, 'Tendance (°C/an)': round(slope, 3)})

tendances = pd.DataFrame(resultats).sort_values('Tendance (°C/an)', ascending=False)

# Visualisation
plt.figure(figsize=(12, 6))
sns.barplot(data=tendances, x='Ville', y='Tendance (°C/an)')
plt.title('Tendance de Réchauffement par Ville')
plt.xticks(rotation=45)
plt.show()

display(tendances)

## 4. Classification Intégrée

In [ ]:
# Création d'un score climatique
classement = df.groupby('ville').agg({
    'temperature': 'mean',
    'amplitude_thermique': 'mean',
    'precipitation': 'mean'
})

# Normalisation
classement_norm = (classement - classement.mean()) / classement.std()

# Visualisation radar
from math import pi

categories = classement_norm.columns
N = len(categories)

angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]

plt.figure(figsize=(10, 10))
ax = plt.subplot(111, polar=True)
ax.set_theta_offset(pi / 2)
ax.set_theta_direction(-1)

for ville in classement_norm.index:
    values = classement_norm.loc[ville].values.flatten().tolist()
    values += values[:1]
    ax.plot(angles, values, linewidth=1, linestyle='solid', label=ville)
    ax.fill(angles, values, alpha=0.1)

plt.xticks(angles[:-1], categories)
plt.title('Profil Climatique Comparé', y=1.1)
plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
plt.show()

## 5. Conclusions

**Réponses à la problématique** :

1. **Évolution climatique** :
   - Tendance au réchauffement la plus marquée à [Ville] (+X°C/an)
   - [Ville] montre la plus grande stabilité interannuelle

2. **Classement des villes** :
   - **Stabilité** : 1) [Ville1] (σ=X°C), 2) [Ville2], ...
   - **Chaleur** : 1) [VilleA] (moy=X°C), 2) [VilleB], ...
   - **Variabilité** : 1) [VilleX] (amplitude=Y°C), ...

3. **Insights clés** :
   - Correlation négative entre température et précipitations (r=-0.XX)
   - [Ville] présente un profil climatique unique dû à [caractéristique]